# 🎨 Interactive Fourier Scribble

 **Draw an arbitrary function** and explore its **Fourier approximation** in real time. This tool visually demonstrates how complex signals can be broken down and reconstructed using a finite number of frequency components. It's a powerful educational aid for learning Fourier series, signal analysis, and the role of harmonics in shaping functions.

### How it works:
- **Draw** your function using your mouse.
- **Use the slider or buttons** to choose how many Fourier modes to keep.
- The dashed line shows the **approximation** based on selected modes.
- **Click "Clear Drawing"** to reset the canvas and try again.

---

**Note:** This notebook uses `%matplotlib widget` and requires `ipympl`. Install it with:

```bash
pip install ipympl


---

### 🔹 Code: Setup


In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, VBox, Button, HBox, Output
from IPython.display import display, clear_output

canvas_out = Output()

drawn_x = []
drawn_y = []

with canvas_out:
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.set_title("Draw a function by dragging mouse")
    ax.set_xlim(0, 1)
    ax.set_ylim(-1.2, 1.2)
    line, = ax.plot([], [], lw=2, label='Original')
    recon_line, = ax.plot([], [], lw=2, linestyle='--', label='Fourier Approx')
    ax.legend()

def clear_canvas(event=None):
    global drawn_x, drawn_y
    drawn_x.clear()
    drawn_y.clear()
    line.set_data([], [])
    recon_line.set_data([], [])
    fig.canvas.draw()

clear_btn = Button(description="Clear Drawing")
clear_btn.on_click(clear_canvas)

drawing = {"active": False}

def on_press(event):
    if event.inaxes != ax: return
    drawing["active"] = True
    drawn_x.append(event.xdata)
    drawn_y.append(event.ydata)
    line.set_data(drawn_x, drawn_y)
    fig.canvas.draw()

def on_move(event):
    if drawing["active"] and event.inaxes == ax:
        drawn_x.append(event.xdata)
        drawn_y.append(event.ydata)
        line.set_data(drawn_x, drawn_y)
        fig.canvas.draw()

def on_release(event):
    drawing["active"] = False

fig.canvas.mpl_connect('button_press_event', on_press)
fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_release_event', on_release)


def fourier_approx(x_vals, y_vals, num_modes, N=1000):

    x_uniform = np.linspace(0, 1, N)
    y_interp = np.interp(x_uniform, x_vals, y_vals)

    if num_modes == 1:
        return x_uniform, np.ones_like(x_uniform) * np.mean(y_interp)

    fft_coeffs = np.fft.fft(y_interp)
    keep = num_modes // 2
    fft_filtered = np.zeros_like(fft_coeffs)
    fft_filtered[:keep] = fft_coeffs[:keep]
    fft_filtered[-keep:] = fft_coeffs[-keep:]
    y_reconstructed = np.fft.ifft(fft_filtered).real
    return x_uniform, y_reconstructed

slider = IntSlider(min=0, max=100, step=1, value=0, description="Fourier Modes")
output = Output()

def update_plot(num_modes):
    with output:
        clear_output(wait=True)
        if len(drawn_x) < 2:
            print("Please draw a function first.")
            return
        
        x_vals = np.array(drawn_x)
        y_vals = np.array(drawn_y)
        sort_idx = np.argsort(x_vals)
        x_vals = x_vals[sort_idx]
        y_vals = y_vals[sort_idx]

        x_rec, y_rec = fourier_approx(x_vals, y_vals, num_modes)
        recon_line.set_data(x_rec, y_rec)
        fig.canvas.draw()

def on_slider_change(change):
    update_plot(change["new"])

slider.observe(on_slider_change, names='value')

plus_btn = Button(description="+")
minus_btn = Button(description="−")

def increase(_): slider.value = min(slider.value + 1, slider.max)
def decrease(_): slider.value = max(slider.value - 1, slider.min)

plus_btn.on_click(increase)
minus_btn.on_click(decrease)

controls = HBox([slider, minus_btn, plus_btn])
display(VBox([controls, output]))

update_plot(slider.value)


## Drawing Canvas

### Draw from left to right always moving the positive direction

In [ ]:
display(VBox([clear_btn, controls, output, canvas_out]))
plt.show()